In [1]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

env = make_vec_env(
    "HalfCheetah-v4",
    rng=np.random.default_rng(),
    n_envs=4,
    env_make_kwargs={"render_mode": "rgb_array"},
)


expert = load_policy(
    "sac-huggingface",
    organization="sb3",
    env_name="HalfCheetah-v3",
    venv=env,
)

C:\Users\xavier\anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: 'bytes' object cannot be interpreted as an integer
  warnings.warn(
C:\Users\xavier\anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend transitioning to Gymnasium by saving that model again.
  warnings.warn(


In [2]:
from src.models.dagger import TrajectoryAccumulator
from src.models.dagger import generate_trajectories

In [3]:
obs = env.reset()
print(obs.shape)
print(env.num_envs,env.observation_space.shape[0] ,env.action_space.shape[0])

(4, 17)
4 17 6


In [4]:
traj = generate_trajectories(expert, env)

partial_trajectories {0: [], 1: [], 2: [], 3: []}
Start generate Traj
finish_trajectory
finish_trajectory
finish_trajectory
finish_trajectory
active env sum  0
active env [False False False False]
END


In [5]:
print(len(traj[0]))
print(len(traj[1]))
print(len(traj[2]))
print(len(traj[3]))

1001
1001
1001
1001


In [6]:
print(traj[0][44])
print(traj[1][44])

{'acts': array([-0.5271554 ,  0.30056047, -0.30244225, -0.5404374 , -0.85006374,
       -0.42319733], dtype=float32), 'rews': -0.16523148, 'next_obs': array([-2.42663704e-01,  2.10417418e-01, -2.38853508e-01,  2.11678274e-01,
       -1.78653869e-01, -4.13961262e-01, -5.37901678e-01, -2.34730622e-01,
        1.09122265e-04, -9.92794956e-05, -5.09333508e-05,  5.97378926e-04,
        4.56548483e-04,  5.80608531e-04, -3.35490091e-04, -1.04279205e-04,
        4.10465653e-04]), 'dones': False}
{'acts': array([-0.6864851 ,  0.03523588, -0.2958427 , -0.6306479 , -0.9497216 ,
       -0.7521548 ], dtype=float32), 'rews': -0.24229474, 'next_obs': array([-2.29599610e-01,  2.52135061e-01, -3.35384931e-01,  6.05665469e-02,
       -1.97162535e-01, -4.60677934e-01, -5.65993187e-01, -3.64459318e-01,
        2.52118486e-04, -9.54029932e-05,  7.17936563e-05,  8.50477811e-04,
        7.07578878e-04,  7.34645907e-04, -5.82745561e-04, -3.05304971e-04,
       -1.72039684e-05]), 'dones': False}


In [7]:
combined_trajectories = []

for t in traj:
    combined_trajectories.extend(t)

print(len(combined_trajectories))

4004


In [8]:
from src.models.dagger import ReplayBuffer
from src.models.dagger import BC
from src.models.dagger import SimpleDAggerTrainer

In [9]:
rb = ReplayBuffer(combined_trajectories)

In [10]:
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [11]:
rb_loader = DataLoader(rb, batch_size=32, shuffle=True)

In [12]:
for i, batch in enumerate(tqdm(rb_loader), 0):
    print(batch[0].shape,batch[1].shape )
    break

  0%|                                                                                          | 0/126 [00:00<?, ?it/s]

torch.Size([32, 17]) torch.Size([32, 6])


In [13]:
bc_trainer = BC(
    observation_space=env.observation_space, 
    action_space=env.action_space, 
    rng=np.random.default_rng(),
    demonstrations=rb)

observation_space Box(-inf, inf, (17,), float64)


In [14]:
dagger_trainer = SimpleDAggerTrainer(
    venv=env,
    expert_policy=expert,
    bc_trainer=bc_trainer,
    rng=np.random.default_rng(),
)